In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import plotly.graph_objects as go

In [3]:
file_list = ['new_ele_data.csv','new_mid_data.csv','new_high_data.csv']

In [4]:
df = pd.DataFrame()
for file in file_list:
    tmp_df = pd.read_csv(file,encoding='cp949')
    df = pd.concat([df,tmp_df],axis=0)

In [5]:
new_ele_data = pd.read_csv('new_ele_data.csv',encoding='cp949')
new_mid_data = pd.read_csv('new_mid_data.csv',encoding='cp949')
new_high_data = pd.read_csv('new_high_data.csv',encoding='cp949')

In [6]:
new_ele_data.columns = new_ele_data.columns.str.replace(' ','')
new_mid_data.columns = new_ele_data.columns.str.replace(' ','')
new_high_data.columns = new_ele_data.columns.str.replace(' ','')

In [7]:
df = pd.concat([new_ele_data,new_mid_data,new_high_data],axis=0)

In [8]:
df.year = df.year.astype('str')

In [9]:
df.year = df.year.str.slice(0,4)

In [10]:
html_title = html.H3('지역별 초중고 교사 1명당 학생 수 변화')
school_select_radio = dcc.RadioItems(
    id = 'school-select-radio',
    options = [{'label':i, 'value':i} for i in df.speices.unique()]
    
)
local_select_dropdown = dcc.Dropdown(
    id = 'local-select-dropdown',
    options = [{'label':i, 'value':i} for i in df.country.unique()]
)
school_select_radio1 = dcc.RadioItems(
    id = 'school-select-radio1',
    options = [{'label':i, 'value':i} for i in df.speices.unique()]
)
year_select_dropdown = dcc.Dropdown(
    id = 'year-select-dropdown',
    options = [{'label':i, 'value':i} for i in df.year.unique()]
)


In [11]:
app = dash.Dash(__name__)
app.layout = html.Div(
    [
        html_title,
        html.Hr(),
        html.Label('학교선택'),
        school_select_radio,
        html.Label('지역선택'),
        local_select_dropdown,
        html.Hr(),
        dcc.Graph(
            id = 'school-local-line-graph'
        ),
        html.Hr(),
        html.Label('학교선택'),
        school_select_radio1,
        html.Label('년도선택'),
        year_select_dropdown,
        dcc.Graph(
            id = 'school-year-bar-graph'
        )
    ]
)

@app.callback(
    Output('school-local-line-graph','figure'),
    Input('school-select-radio','value'),
    Input('local-select-dropdown','value')
)
def update_graph(school,local):
    dt = df.loc[(df.speices==school) & (df.country == local),['year','mean_student_per_teacher']]
    fig = go.Figure(data=go.Scatter(x=dt.year, y=dt.mean_student_per_teacher))
    return fig

@app.callback(
    Output('school-year-bar-graph','figure'),
    Input('school-select-radio1','value'),
    Input('year-select-dropdown','value')
)

def update_bar_graph(school,year):
    dt = df.loc[(df.speices==school) & (df.year == year),['country','mean_student_per_teacher']]
    colors = ['lightslategray',] * dt.country.size
    colors[np.argmax(dt.mean_student_per_teacher)] = 'crimson'
    fig = go.Figure(data=[go.Bar(x=dt.country,y=dt.mean_student_per_teacher,marker_color=colors )])
    return fig


In [ ]:
app.run_server(port='7777')

Dash is running on http://127.0.0.1:7777/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:7777/ (Press CTRL+C to quit)
